In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

In [2]:
print("cwd:", os.getcwd())

ROOT = Path.cwd()
if not (ROOT / "data" / "raw" / "secom.data").exists():
    ROOT = ROOT.parent  # handles running from notebooks/

x_path = ROOT / "data" / "raw" / "secom.data"
y_path = ROOT / "data" / "raw" / "secom_labels.data"

X = pd.read_csv(x_path, sep=r"\s+", header=None, na_values=["NaN", "nan"])
L = pd.read_csv(y_path, sep=r"\s+", header=None)

# Handle label/timestamp format variants
if L.shape[1] == 3:
    L.columns = ["y", "date", "time"]
    L["timestamp"] = pd.to_datetime(L["date"] + " " + L["time"], errors="coerce")
elif L.shape[1] == 2:
    L.columns = ["y", "timestamp"]
    L["timestamp"] = pd.to_datetime(L["timestamp"], format="%d/%m/%Y %H:%M:%S", errors="coerce")
else:
    raise ValueError(f"Unexpected label file shape: {L.shape}")

print("X shape:", X.shape)
print("L shape:", L.shape)
print("Row counts match:", len(X) == len(L))
print("Feature count:", X.shape[1])
print("Unique labels:", sorted(L["y"].dropna().unique().tolist()))
print("Label counts:\n", L["y"].value_counts(dropna=False))
print("Timestamp parse success:", L["timestamp"].notna().mean())
print("Head labels+time:\n", L[["y", "timestamp"]].head())

cwd: e:\GitHub\Mini-Projects\secom-yield-monitoring\notebooks
X shape: (1567, 590)
L shape: (1567, 2)
Row counts match: True
Feature count: 590
Unique labels: [-1, 1]
Label counts:
 y
-1    1463
 1     104
Name: count, dtype: int64
Timestamp parse success: 1.0
Head labels+time:
    y           timestamp
0 -1 2008-07-19 11:55:00
1 -1 2008-07-19 12:32:00
2  1 2008-07-19 13:17:00
3 -1 2008-07-19 14:43:00
4 -1 2008-07-19 15:22:00


In [3]:
y = (L["y"] == 1).astype(int)  # 1=fail, 0=pass

miss = X.isna().mean()
print("Missingness summary:")
print(miss.describe())

for t in [0.2, 0.4, 0.6, 0.8, 0.95]:
    print(f"cols with >{int(t*100)}% missing: {(miss > t).sum()}")

miss_by_class = pd.DataFrame({
    "pass_missing": X[y == 0].isna().mean(),
    "fail_missing": X[y == 1].isna().mean(),
})
miss_by_class["delta_fail_minus_pass"] = miss_by_class["fail_missing"] - miss_by_class["pass_missing"]

print("\nTop 15 features where missingness differs by class:")
print(
    miss_by_class["delta_fail_minus_pass"]
    .abs()
    .sort_values(ascending=False)
    .head(15)
)

Missingness summary:
count    590.000000
mean       0.045375
std        0.154340
min        0.000000
25%        0.001276
50%        0.003829
75%        0.005743
max        0.911934
dtype: float64
cols with >20% missing: 32
cols with >40% missing: 32
cols with >60% missing: 24
cols with >80% missing: 8
cols with >95% missing: 0

Top 15 features where missingness differs by class:
72     0.171960
73     0.171960
345    0.171960
346    0.171960
385    0.148858
112    0.148858
519    0.148858
247    0.148858
111    0.066289
109    0.066289
382    0.066289
110    0.066289
516    0.066289
244    0.066289
246    0.066289
Name: delta_fail_minus_pass, dtype: float64


In [4]:
df = L.copy()
df["fail"] = (df["y"] == 1).astype(int)
df = df.sort_values("timestamp")

print("time range:", df["timestamp"].min(), "->", df["timestamp"].max())

weekly = (
    df.set_index("timestamp")["fail"]
      .resample("W")
      .agg(["count", "sum"])
      .rename(columns={"sum": "fails"})
) # type: ignore
weekly["fail_rate"] = weekly["fails"] / weekly["count"]

print("\nweekly fail-rate summary:")
print(weekly["fail_rate"].describe())

print("\nTop 10 highest-fail weeks:")
print(weekly.sort_values("fail_rate", ascending=False).head(10))

print("\nTop 10 lowest-fail weeks:")
print(weekly.sort_values("fail_rate", ascending=True).head(10))

time range: 2008-07-19 11:55:00 -> 2008-10-17 06:07:00

weekly fail-rate summary:
count    14.000000
mean      0.087106
std       0.069633
min       0.010638
25%       0.034706
50%       0.073364
75%       0.123665
max       0.230769
Name: fail_rate, dtype: float64

Top 10 highest-fail weeks:
            count  fails  fail_rate
timestamp                          
2008-07-20     13      3   0.230769
2008-08-03     48     10   0.208333
2008-08-17     51      7   0.137255
2008-08-10    108     14   0.129630
2008-08-24    208     22   0.105769
2008-07-27     21      2   0.095238
2008-10-05    169     15   0.088757
2008-10-12    138      8   0.057971
2008-09-14     95      4   0.042105
2008-08-31    169      7   0.041420

Top 10 lowest-fail weeks:
            count  fails  fail_rate
timestamp                          
2008-10-19     94      1   0.010638
2008-09-07    133      2   0.015038
2008-09-28    166      4   0.024096
2008-09-21    154      5   0.032468
2008-08-31    169      7   0.04

In [5]:
# X already loaded
n = len(X)

# 1) Constant / near-constant
nunique = X.nunique(dropna=True)
const_cols = nunique[nunique <= 1].index.tolist()

# near-constant by dominant value frequency (ignoring NaN)
dom_frac = X.apply(lambda s: s.value_counts(dropna=True, normalize=True).iloc[0] if s.notna().any() else 1.0)
near_const_cols = dom_frac[dom_frac >= 0.995].index.tolist()

print("constant features:", len(const_cols))
print("near-constant (>=99.5% same value):", len(near_const_cols))

# 2) Correlation redundancy (after median impute only for this audit)
Xi = X.copy()
Xi = Xi.fillna(Xi.median(numeric_only=True))

corr = Xi.corr().abs()
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
high_corr_pairs = (upper.stack().sort_values(ascending=False))

print("\n# pairs with |corr| >= 0.95:", int((high_corr_pairs >= 0.95).sum()))
print("# pairs with |corr| >= 0.90:", int((high_corr_pairs >= 0.90).sum()))
print("\nTop 20 absolute-correlation pairs:")
print(high_corr_pairs.head(20))

constant features: 116
near-constant (>=99.5% same value): 122

# pairs with |corr| >= 0.95: 316
# pairs with |corr| >= 0.90: 397

Top 20 absolute-correlation pairs:
209  347    1.000000
     342    1.000000
     478    1.000000
74   478    1.000000
     209    1.000000
     342    1.000000
     347    1.000000
342  347    1.000000
347  478    1.000000
206  209    1.000000
     347    1.000000
     478    1.000000
74   206    1.000000
206  342    1.000000
342  478    1.000000
34   36     1.000000
140  275    1.000000
172  174    1.000000
307  309    0.999999
152  287    0.999997
dtype: float64


In [6]:
y_bin = (L["y"] == 1).astype(int)

# base masks
miss = X.isna().mean()
dom = X.apply(lambda s: s.value_counts(dropna=True, normalize=True).iloc[0] if s.notna().any() else 1.0)
nunique = X.nunique(dropna=True)

drop_const = set(X.columns[nunique <= 1])
drop_near_const = set(X.columns[dom >= 0.995])

base_keep = [c for c in X.columns if c not in drop_const and c not in drop_near_const]
Xi = X[base_keep].copy().fillna(X[base_keep].median())

# high-corr duplicate graph
corr = Xi.corr().abs()
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
pairs = upper.stack()
dup_pairs = pairs[pairs >= 0.9999].reset_index()
dup_pairs.columns = ["a", "b", "abs_corr"]

# adjacency
adj = {c: set() for c in Xi.columns}
for a, b, _ in dup_pairs.itertuples(index=False):
    adj[a].add(b)
    adj[b].add(a)

# connected components
seen, comps = set(), []
for n in Xi.columns:
    if n in seen or not adj[n]:
        continue
    stack, comp = [n], set()
    while stack:
        v = stack.pop()
        if v in seen:
            continue
        seen.add(v)
        comp.add(v)
        stack.extend(adj[v] - seen)
    comps.append(comp)

# representative chooser
var = Xi.var()
yc = y_bin - y_bin.mean()
rep_keep, rep_drop = set(), set()

for comp in comps:
    comp = list(comp)
    # rank: lower missing, then higher variance, then higher |corr with y|
    cxy = {}
    for c in comp:
        xc = Xi[c] - Xi[c].mean()
        cxy[c] = abs((xc @ yc) / (np.linalg.norm(xc) * np.linalg.norm(yc) + 1e-12))
    best = sorted(comp, key=lambda c: (miss[c], -var[c], -cxy[c]))[0]
    rep_keep.add(best)
    rep_drop.update(set(comp) - {best})

final_drop = drop_const | drop_near_const | rep_drop
final_keep = [c for c in X.columns if c not in final_drop]

print("drop_const:", len(drop_const))
print("drop_near_const:", len(drop_near_const))
print("duplicate_components:", len(comps))
print("drop_from_duplicates:", len(rep_drop))
print("final_keep_count:", len(final_keep))

drop_const: 116
drop_near_const: 122
duplicate_components: 10
drop_from_duplicates: 12
final_keep_count: 456


In [7]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import SelectKBest, f_classif, r_regression

In [8]:
X_base = X[final_keep].copy()
y_bin = (L["y"] == 1).astype(int).values  # 1=fail, 0=pass

def eval_baseline(add_indicator=True):
    pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="median", add_indicator=add_indicator)),
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(
            solver="lbfgs",
            class_weight="balanced",
            max_iter=3000,
            random_state=42,
        )),
    ])

    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

    rows = []
    for fold, (tr, te) in enumerate(cv.split(X_base, y_bin), start=1):
        Xtr, Xte = X_base.iloc[tr], X_base.iloc[te]
        ytr, yte = y_bin[tr], y_bin[te]

        pipe.fit(Xtr, ytr)
        pred = pipe.predict(Xte)

        tn, fp, fn, tp = confusion_matrix(yte, pred, labels=[0, 1]).ravel()
        tpr = tp / (tp + fn + 1e-12)   # True+
        tnr = tn / (tn + fp + 1e-12)   # True-
        ber = 1.0 - 0.5 * (tpr + tnr)

        rows.append({"fold": fold, "BER": ber, "True+": tpr, "True-": tnr})

    out = pd.DataFrame(rows)
    print(f"\nadd_indicator={add_indicator}")
    print(out[["BER","True+","True-"]].mean().rename("mean"))
    print(out[["BER","True+","True-"]].std().rename("std"))
    return out

In [9]:
class S2NSelector(BaseEstimator, TransformerMixin):
    def __init__(self, k=40, eps=1e-12):
        self.k = k
        self.eps = eps

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y)
        pos = X[y == 1]
        neg = X[y == 0]

        mu_pos = np.nanmean(pos, axis=0)
        mu_neg = np.nanmean(neg, axis=0)
        sd_pos = np.nanstd(pos, axis=0, ddof=0)
        sd_neg = np.nanstd(neg, axis=0, ddof=0)

        scores = np.abs(mu_pos - mu_neg) / (sd_pos + sd_neg + self.eps)
        scores = np.nan_to_num(scores, nan=0.0, posinf=0.0, neginf=0.0)

        self.scores_ = scores
        self.idx_ = np.argsort(scores)[::-1][: self.k]
        return self

    def transform(self, X):
        return np.asarray(X)[:, self.idx_]

def run_s2n(k=40):
    pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="median", add_indicator=True)),
        ("scaler", StandardScaler()),
        ("s2n", S2NSelector(k=k)),
        ("clf", LogisticRegression(
            solver="lbfgs",
            class_weight="balanced",
            max_iter=3000,
            random_state=42,
        )),
    ])

    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    rows = []
    selected_counts = {}

    for fold, (tr, te) in enumerate(cv.split(X_base, y_bin), start=1):
        Xtr, Xte = X_base.iloc[tr], X_base.iloc[te]
        ytr, yte = y_bin[tr], y_bin[te]

        pipe.fit(Xtr, ytr)
        pred = pipe.predict(Xte)

        tn, fp, fn, tp = confusion_matrix(yte, pred, labels=[0,1]).ravel()
        tpr = tp / (tp + fn + 1e-12)
        tnr = tn / (tn + fp + 1e-12)
        ber = 1 - 0.5 * (tpr + tnr)

        rows.append({"fold": fold, "BER": ber, "True+": tpr, "True-": tnr})

        idx = pipe.named_steps["s2n"].idx_
        for j in idx:
            selected_counts[j] = selected_counts.get(j, 0) + 1

    out = pd.DataFrame(rows)
    print(out[["BER","True+","True-"]].mean().rename("mean"))
    print(out[["BER","True+","True-"]].std().rename("std"))

    freq = pd.Series(selected_counts).sort_values(ascending=False) / 10.0
    print("\nTop selected transformed columns (frequency across folds):")
    print(freq.head(20))
    return out, freq


In [10]:
class WelchTSelector(BaseEstimator, TransformerMixin):
    def __init__(self, k=40, eps=1e-12):
        self.k = k
        self.eps = eps

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y)

        pos = X[y == 1]
        neg = X[y == 0]

        m1 = np.mean(pos, axis=0)
        m0 = np.mean(neg, axis=0)
        v1 = np.var(pos, axis=0, ddof=1)
        v0 = np.var(neg, axis=0, ddof=1)
        n1 = max(pos.shape[0], 1)
        n0 = max(neg.shape[0], 1)

        t = np.abs(m1 - m0) / (np.sqrt(v1 / n1 + v0 / n0) + self.eps)
        t = np.nan_to_num(t, nan=0.0, posinf=0.0, neginf=0.0)

        self.scores_ = t
        self.idx_ = np.argsort(t)[::-1][: self.k]
        return self

    def transform(self, X):
        return np.asarray(X)[:, self.idx_]


def run_t(k=40, random_state=42):
    pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="median", add_indicator=True)),
        ("scaler", StandardScaler()),
        ("tsel", WelchTSelector(k=k)),
        ("clf", LogisticRegression(
            solver="lbfgs",
            class_weight="balanced",
            max_iter=3000,
            random_state=random_state,
        )),
    ])

    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=random_state)

    rows = []
    selected_counts = {}

    for fold, (tr, te) in enumerate(cv.split(X_base, y_bin), start=1):
        Xtr, Xte = X_base.iloc[tr], X_base.iloc[te]
        ytr, yte = y_bin[tr], y_bin[te]

        pipe.fit(Xtr, ytr)
        pred = pipe.predict(Xte)

        tn, fp, fn, tp = confusion_matrix(yte, pred, labels=[0, 1]).ravel()
        tpr = tp / (tp + fn + 1e-12)  # True+
        tnr = tn / (tn + fp + 1e-12)  # True-
        ber = 1.0 - 0.5 * (tpr + tnr)

        rows.append({"fold": fold, "BER": ber, "True+": tpr, "True-": tnr})

        idx = pipe.named_steps["tsel"].idx_
        for j in idx:
            selected_counts[j] = selected_counts.get(j, 0) + 1

    out = pd.DataFrame(rows)
    print(out[["BER", "True+", "True-"]].mean().rename("mean"))
    print(out[["BER", "True+", "True-"]].std().rename("std"))

    freq = pd.Series(selected_counts).sort_values(ascending=False) / 10.0
    print("\nTop selected transformed columns (frequency across folds):")
    print(freq.head(20))

    return out, freq

In [11]:
def run_f(k=40, random_state=42):
    pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="median", add_indicator=True)),
        ("scaler", StandardScaler()),
        ("fsel", SelectKBest(score_func=f_classif, k=k)),
        ("clf", LogisticRegression(
            solver="lbfgs",
            class_weight="balanced",
            max_iter=3000,
            random_state=random_state,
        )),
    ])

    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=random_state)
    rows, selected_counts = [], {}

    for fold, (tr, te) in enumerate(cv.split(X_base, y_bin), start=1):
        Xtr, Xte = X_base.iloc[tr], X_base.iloc[te]
        ytr, yte = y_bin[tr], y_bin[te]

        pipe.fit(Xtr, ytr)
        pred = pipe.predict(Xte)

        tn, fp, fn, tp = confusion_matrix(yte, pred, labels=[0, 1]).ravel()
        tpr = tp / (tp + fn + 1e-12)
        tnr = tn / (tn + fp + 1e-12)
        ber = 1.0 - 0.5 * (tpr + tnr)
        rows.append({"fold": fold, "BER": ber, "True+": tpr, "True-": tnr})

        idx = pipe.named_steps["fsel"].get_support(indices=True)
        for j in idx:
            selected_counts[j] = selected_counts.get(j, 0) + 1

    out = pd.DataFrame(rows)
    print(out[["BER", "True+", "True-"]].mean().rename("mean"))
    print(out[["BER", "True+", "True-"]].std().rename("std"))

    freq = pd.Series(selected_counts).sort_values(ascending=False) / 10.0
    print("\nTop selected transformed columns (frequency across folds):")
    print(freq.head(20))
    return out, freq

In [12]:
def run_pearson(k=40, random_state=42):
    pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="median", add_indicator=True)),
        ("scaler", StandardScaler()),
        ("psel", SelectKBest(score_func=lambda X, y: np.abs(r_regression(X, y)), k=k)),
        ("clf", LogisticRegression(
            solver="lbfgs",
            class_weight="balanced",
            max_iter=3000,
            random_state=random_state,
        )),
    ])

    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=random_state)
    rows, selected_counts = [], {}

    for fold, (tr, te) in enumerate(cv.split(X_base, y_bin), start=1):
        Xtr, Xte = X_base.iloc[tr], X_base.iloc[te]
        ytr, yte = y_bin[tr], y_bin[te]

        pipe.fit(Xtr, ytr)
        pred = pipe.predict(Xte)

        tn, fp, fn, tp = confusion_matrix(yte, pred, labels=[0, 1]).ravel()
        tpr = tp / (tp + fn + 1e-12)
        tnr = tn / (tn + fp + 1e-12)
        ber = 1.0 - 0.5 * (tpr + tnr)
        rows.append({"fold": fold, "BER": ber, "True+": tpr, "True-": tnr})

        idx = pipe.named_steps["psel"].get_support(indices=True)
        for j in idx:
            selected_counts[j] = selected_counts.get(j, 0) + 1

    out = pd.DataFrame(rows)
    print(out[["BER", "True+", "True-"]].mean().rename("mean"))
    print(out[["BER", "True+", "True-"]].std().rename("std"))

    freq = pd.Series(selected_counts).sort_values(ascending=False) / 10.0
    print("\nTop selected transformed columns (frequency across folds):")
    print(freq.head(20))
    return out, freq

In [13]:
from skrebate import ReliefF

class ReliefFSelector(BaseEstimator, TransformerMixin):
    def __init__(self, k=40, n_neighbors=10):
        self.k = k
        self.n_neighbors = n_neighbors

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y)
        self.rf_ = ReliefF(
            n_features_to_select=self.k,
            n_neighbors=self.n_neighbors,
        )
        self.rf_.fit(X, y)

        imp = np.asarray(self.rf_.feature_importances_, dtype=float)
        imp = np.nan_to_num(imp, nan=-np.inf)
        self.scores_ = imp
        self.idx_ = np.argsort(imp)[::-1][: self.k]
        return self

    def transform(self, X):
        return np.asarray(X)[:, self.idx_]


def run_relieff(k=40, n_neighbors=10, random_state=42):
    pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="median", add_indicator=True)),
        ("scaler", StandardScaler()),
        ("rsel", ReliefFSelector(k=k, n_neighbors=n_neighbors)),
        ("clf", LogisticRegression(
            solver="lbfgs",
            class_weight="balanced",
            max_iter=3000,
            random_state=random_state,
        )),
    ])

    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=random_state)

    rows = []
    selected_counts = {}

    for fold, (tr, te) in enumerate(cv.split(X_base, y_bin), start=1):
        Xtr, Xte = X_base.iloc[tr], X_base.iloc[te]
        ytr, yte = y_bin[tr], y_bin[te]

        pipe.fit(Xtr, ytr)
        pred = pipe.predict(Xte)

        tn, fp, fn, tp = confusion_matrix(yte, pred, labels=[0, 1]).ravel()
        tpr = tp / (tp + fn + 1e-12)  # True+
        tnr = tn / (tn + fp + 1e-12)  # True-
        ber = 1.0 - 0.5 * (tpr + tnr)

        rows.append({"fold": fold, "BER": ber, "True+": tpr, "True-": tnr})

        idx = pipe.named_steps["rsel"].idx_
        for j in idx:
            selected_counts[j] = selected_counts.get(j, 0) + 1

    out = pd.DataFrame(rows)
    print(out[["BER", "True+", "True-"]].mean().rename("mean"))
    print(out[["BER", "True+", "True-"]].std().rename("std"))

    freq = pd.Series(selected_counts).sort_values(ascending=False) / 10.0
    print("\nTop selected transformed columns (frequency across folds):")
    print(freq.head(20))

    return out, freq

In [14]:
class GramSchmidtSelector(BaseEstimator, TransformerMixin):
    def __init__(self, k=40, eps=1e-12):
        self.k = k
        self.eps = eps

    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y, dtype=float)

        n, p = X.shape
        k = min(self.k, p)

        # Work copies
        Xw = X.copy()
        r = y - y.mean()  # residual target direction

        remaining = list(range(p))
        selected = []
        scores = []

        for _ in range(k):
            r_norm = np.linalg.norm(r)
            if r_norm < self.eps or not remaining:
                break

            # score remaining features by absolute cosine with residual
            best_j = None
            best_score = -np.inf

            for j in remaining:
                xj = Xw[:, j]
                x_norm = np.linalg.norm(xj)
                if x_norm < self.eps:
                    s = -np.inf
                else:
                    s = abs(np.dot(xj, r)) / (x_norm * r_norm + self.eps)

                if s > best_score:
                    best_score = s
                    best_j = j

            if best_j is None or not np.isfinite(best_score):
                break

            selected.append(best_j)
            scores.append(best_score)

            # orthonormal direction q of selected feature
            q = Xw[:, best_j]
            q_norm = np.linalg.norm(q)
            if q_norm < self.eps:
                remaining.remove(best_j)
                continue
            q = q / q_norm

            # remove selected direction from residual and remaining features
            r = r - np.dot(r, q) * q

            for j in remaining:
                if j == best_j:
                    continue
                Xw[:, j] = Xw[:, j] - np.dot(Xw[:, j], q) * q

            remaining.remove(best_j)

        self.idx_ = np.array(selected, dtype=int)
        self.scores_ = np.array(scores, dtype=float)

        # pad if early stop (rare)
        if len(self.idx_) < k:
            leftovers = [j for j in range(p) if j not in set(self.idx_)]
            need = k - len(self.idx_)
            self.idx_ = np.concatenate([self.idx_, np.array(leftovers[:need], dtype=int)])

        return self

    def transform(self, X):
        return np.asarray(X)[:, self.idx_]


def run_gram_schmidt(k=40, random_state=42):
    pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="median", add_indicator=True)),
        ("scaler", StandardScaler()),
        ("gsel", GramSchmidtSelector(k=k)),
        ("clf", LogisticRegression(
            solver="lbfgs",
            class_weight="balanced",
            max_iter=3000,
            random_state=random_state,
        )),
    ])

    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=random_state)

    rows = []
    selected_counts = {}

    for fold, (tr, te) in enumerate(cv.split(X_base, y_bin), start=1):
        Xtr, Xte = X_base.iloc[tr], X_base.iloc[te]
        ytr, yte = y_bin[tr], y_bin[te]

        pipe.fit(Xtr, ytr)
        pred = pipe.predict(Xte)

        tn, fp, fn, tp = confusion_matrix(yte, pred, labels=[0, 1]).ravel()
        tpr = tp / (tp + fn + 1e-12)  # True+
        tnr = tn / (tn + fp + 1e-12)  # True-
        ber = 1.0 - 0.5 * (tpr + tnr)

        rows.append({"fold": fold, "BER": ber, "True+": tpr, "True-": tnr})

        idx = pipe.named_steps["gsel"].idx_
        for j in idx:
            selected_counts[j] = selected_counts.get(j, 0) + 1

    out = pd.DataFrame(rows)
    print(out[["BER", "True+", "True-"]].mean().rename("mean"))
    print(out[["BER", "True+", "True-"]].std().rename("std"))

    freq = pd.Series(selected_counts).sort_values(ascending=False) / 10.0
    print("\nTop selected transformed columns (frequency across folds):")
    print(freq.head(20))

    return out, freq

In [15]:
def _metrics_from_pred(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    tpr = tp / (tp + fn + 1e-12)  # True+
    tnr = tn / (tn + fp + 1e-12)  # True-
    ber = 1.0 - 0.5 * (tpr + tnr)
    return ber, tpr, tnr


def run_l1(k=40, C=0.15, random_state=42):
    """
    Embedded selection via L1 logistic.
    Top-k selected each fold by |coef| (on preprocessed train fold).
    """
    pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="median", add_indicator=True)),
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(
            solver="saga",
            C=C,
            l1_ratio=1.0,  # new sklearn style for pure L1
            class_weight="balanced",
            max_iter=8000,
            random_state=random_state,
        )),
    ])

    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=random_state)

    rows = []
    selected_counts = {}

    for fold, (tr, te) in enumerate(cv.split(X_base, y_bin), start=1):
        Xtr, Xte = X_base.iloc[tr], X_base.iloc[te]
        ytr, yte = y_bin[tr], y_bin[te]

        pipe.fit(Xtr, ytr)

        # rank by absolute coefficient, keep top-k
        coef = np.abs(pipe.named_steps["clf"].coef_[0])
        idx = np.argsort(coef)[::-1][:k]

        # transform train/test through preprocessors only, then subset columns
        Xtr_t = pipe[:-1].transform(Xtr)[:, idx]
        Xte_t = pipe[:-1].transform(Xte)[:, idx]

        # refit logistic on selected columns only
        clf2 = LogisticRegression(
            solver="lbfgs",
            class_weight="balanced",
            max_iter=4000,
            random_state=random_state,
        )
        clf2.fit(Xtr_t, ytr)
        pred = clf2.predict(Xte_t)

        ber, tpr, tnr = _metrics_from_pred(yte, pred)
        rows.append({"fold": fold, "BER": ber, "True+": tpr, "True-": tnr})

        for j in idx:
            selected_counts[j] = selected_counts.get(j, 0) + 1

    out = pd.DataFrame(rows)
    print(f"L1 (C={C}, k={k})")
    print(out[["BER", "True+", "True-"]].mean().rename("mean"))
    print(out[["BER", "True+", "True-"]].std().rename("std"))

    freq = pd.Series(selected_counts).sort_values(ascending=False) / 10.0
    print("\nTop selected transformed columns (frequency across folds):")
    print(freq.head(20))
    return out, freq


def run_elasticnet(k=40, C=0.15, l1_ratio=0.4, random_state=42):
    """
    Embedded selection via Elastic Net logistic.
    Top-k selected each fold by |coef| (on preprocessed train fold).
    """
    pipe = Pipeline([
        ("imputer", SimpleImputer(strategy="median", add_indicator=True)),
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(
            solver="saga",
            C=C,
            l1_ratio=l1_ratio,
            class_weight="balanced",
            max_iter=8000,
            random_state=random_state,
        )),
    ])

    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=random_state)

    rows = []
    selected_counts = {}

    for fold, (tr, te) in enumerate(cv.split(X_base, y_bin), start=1):
        Xtr, Xte = X_base.iloc[tr], X_base.iloc[te]
        ytr, yte = y_bin[tr], y_bin[te]

        pipe.fit(Xtr, ytr)

        # rank by absolute coefficient, keep top-k
        coef = np.abs(pipe.named_steps["clf"].coef_[0])
        idx = np.argsort(coef)[::-1][:k]

        # transform train/test through preprocessors only, then subset columns
        Xtr_t = pipe[:-1].transform(Xtr)[:, idx]
        Xte_t = pipe[:-1].transform(Xte)[:, idx]

        # refit logistic on selected columns only
        clf2 = LogisticRegression(
            solver="lbfgs",
            class_weight="balanced",
            max_iter=4000,
            random_state=random_state,
        )
        clf2.fit(Xtr_t, ytr)
        pred = clf2.predict(Xte_t)

        ber, tpr, tnr = _metrics_from_pred(yte, pred)
        rows.append({"fold": fold, "BER": ber, "True+": tpr, "True-": tnr})

        for j in idx:
            selected_counts[j] = selected_counts.get(j, 0) + 1

    out = pd.DataFrame(rows)
    print(f"Elastic Net (C={C}, l1_ratio={l1_ratio}, k={k})")
    print(out[["BER", "True+", "True-"]].mean().rename("mean"))
    print(out[["BER", "True+", "True-"]].std().rename("std"))

    freq = pd.Series(selected_counts).sort_values(ascending=False) / 10.0
    print("\nTop selected transformed columns (frequency across folds):")
    print(freq.head(20))
    return out, freq

In [16]:
class DropAllNaNCols(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        X = np.asarray(X, dtype=float)
        self.keep_idx_ = np.where(~np.all(np.isnan(X), axis=0))[0]
        return self

    def transform(self, X):
        X = np.asarray(X, dtype=float)
        return X[:, self.keep_idx_]

def ber_tpr_tnr(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    tpr = tp / (tp + fn + 1e-12)
    tnr = tn / (tn + fp + 1e-12)
    ber = 1.0 - 0.5 * (tpr + tnr)
    return ber, tpr, tnr

def anchored_splits(n, min_train, test_size, step):
    out = []
    tr_end = min_train
    while tr_end + test_size <= n:
        out.append((0, tr_end, tr_end, tr_end + test_size))
        tr_end += step
    return out

def build_selector(method, k):
    if method == "S2N":
        return S2NSelector(k=k)
    if method == "Welch-t":
        return WelchTSelector(k=k)
    if method == "F-test":
        return SelectKBest(score_func=f_classif, k=k)
    if method == "Pearson":
        # optional: skip because usually same as F-test in this binary setup
        return SelectKBest(score_func=f_classif, k=k)
    if method == "ReliefF":
        return ReliefFSelector(k=k, n_neighbors=10)
    if method == "Gram-Schmidt":
        return GramSchmidtSelector(k=k)
    raise ValueError(method)

def eval_once(Xtr, ytr, Xte, yte, method, k, seed=42):
    pipe = Pipeline([
        ("drop_all_nan", DropAllNaNCols()),
        ("imputer", SimpleImputer(strategy="median", add_indicator=True)),
        ("scaler", StandardScaler()),
        ("sel", build_selector(method, k)),
        ("clf", LogisticRegression(
            solver="lbfgs",
            class_weight="balanced",
            max_iter=3000,
            random_state=seed
        )),
    ])
    pipe.fit(Xtr, ytr)
    pred = pipe.predict(Xte)
    return ber_tpr_tnr(yte, pred)

# --- chronological order ---
ts = pd.to_datetime(L["timestamp"], format="%d/%m/%Y %H:%M:%S", errors="coerce")
valid = ts.notna().values

Xv = X_base.iloc[valid].copy()
yv = y_bin[valid]
tv = ts.iloc[valid].reset_index(drop=True)

order = np.argsort(tv.values)
Xv = Xv.iloc[order].reset_index(drop=True)
yv = yv[order]

# --- config ---
methods = ["S2N", "Welch-t", "F-test", "ReliefF", "Gram-Schmidt"]  # add/remove as needed
k_grid = [10, 20, 40, 60, 80]
outer = anchored_splits(
    n=len(Xv),
    min_train=int(0.50 * len(Xv)),
    test_size=int(0.15 * len(Xv)),
    step=max(30, int(0.15 * len(Xv)) // 3),
)

rows = []

for split_id, (tr_s, tr_e, te_s, te_e) in enumerate(outer, start=1):
    Xtr_outer, ytr_outer = Xv.iloc[tr_s:tr_e], yv[tr_s:tr_e]
    Xte_outer, yte_outer = Xv.iloc[te_s:te_e], yv[te_s:te_e]

    # inner time-aware splits inside outer-train
    n_in = len(Xtr_outer)
    inner = anchored_splits(
        n=n_in,
        min_train=max(200, int(0.60 * n_in)),
        test_size=max(60, int(0.20 * n_in)),
        step=max(20, int(0.20 * n_in) // 2),
    )

    for method in methods:
        k_to_inner_ber = {}

        for k in k_grid:
            inner_bers = []
            for (i_tr_s, i_tr_e, i_va_s, i_va_e) in inner:
                Xi_tr = Xtr_outer.iloc[i_tr_s:i_tr_e]
                yi_tr = ytr_outer[i_tr_s:i_tr_e]
                Xi_va = Xtr_outer.iloc[i_va_s:i_va_e]
                yi_va = ytr_outer[i_va_s:i_va_e]

                if len(np.unique(yi_tr)) < 2 or len(np.unique(yi_va)) < 2:
                    continue

                ber, _, _ = eval_once(Xi_tr, yi_tr, Xi_va, yi_va, method, k)
                inner_bers.append(ber)

            if inner_bers:
                k_to_inner_ber[k] = float(np.mean(inner_bers))

        if not k_to_inner_ber:
            continue

        best_k = min(k_to_inner_ber, key=k_to_inner_ber.get)
        ber, tpr, tnr = eval_once(Xtr_outer, ytr_outer, Xte_outer, yte_outer, method, best_k)

        rows.append({
            "split_id": split_id,
            "method": method,
            "best_k": best_k,
            "inner_BER": k_to_inner_ber[best_k],
            "outer_BER": ber,
            "outer_True+": tpr,
            "outer_True-": tnr,
        })

res = pd.DataFrame(rows)
display(res.head(20))

summary = (
    res.groupby("method")[["outer_BER", "outer_True+", "outer_True-"]]
    .agg(["mean", "std", "median"])
)
display(summary)

k_usage = res.groupby(["method", "best_k"]).size().unstack(fill_value=0)
display(k_usage)

,split_id,method,best_k,inner_BER,outer_BER,outer_True+,outer_True-
0,1,S2N,20,0.324261,0.537611,0.000000,0.924779
1,1,Welch-t,60,0.412291,0.441740,0.333333,0.783186
2,1,F-test,20,0.291447,0.519912,0.000000,0.960177
3,1,ReliefF,80,0.252505,0.477384,0.222222,0.823009
4,1,Gram-Schmidt,20,0.226113,0.555310,0.000000,0.889381
5,2,S2N,40,0.361561,0.603070,0.000000,0.793860
6,2,Welch-t,60,0.472557,0.518484,0.142857,0.820175
7,2,F-test,80,0.365577,0.498747,0.142857,0.859649
8,2,ReliefF,80,0.304315,0.633772,0.000000,0.732456
9,2,Gram-Schmidt,60,0.419220,0.520677,0.142857,0.815789


outer_BER                     outer_True+                      \
                  mean       std    median        mean       std    median   
method                                                                       
F-test        0.458857  0.102047  0.446175    0.235229  0.198248  0.242647   
Gram-Schmidt  0.495218  0.078454  0.516903    0.134449  0.142689  0.119048   
ReliefF       0.512753  0.094685  0.509938    0.205951  0.204358  0.199346   
S2N           0.488631  0.058613  0.471697    0.156549  0.167683  0.145455   
Welch-t       0.478145  0.059148  0.447114    0.179499  0.130117  0.188235   

             outer_True-                      
                    mean       std    median  
method                                        
F-test          0.847057  0.073851  0.859048  
Gram-Schmidt    0.875115  0.051631  0.880811  
ReliefF         0.768542  0.102727  0.734193  
S2N             0.866188  0.105346  0.889327  
Welch-t         0.864211  0.073677  0.837028

best_k,10,20,40,60,80
method,,,,,
F-test,1,3,1,1,2
Gram-Schmidt,0,4,1,2,1
ReliefF,1,0,1,1,5
S2N,1,2,2,1,2
Welch-t,2,0,1,3,2


In [25]:
# ---------- utilities ----------
def ber_tpr_tnr(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    tpr = tp / (tp + fn + 1e-12)  # True+
    tnr = tn / (tn + fp + 1e-12)  # True-
    ber = 1.0 - 0.5 * (tpr + tnr)
    return ber, tpr, tnr

def best_threshold_by_ber(y_true, p_true, grid=None):
    if grid is None:
        grid = np.linspace(0.05, 0.95, 91)
    best_t, best_ber = 0.5, np.inf
    for t in grid:
        pred = (p_true >= t).astype(int)
        ber, _, _ = ber_tpr_tnr(y_true, pred)
        if ber < best_ber:
            best_ber = ber
            best_t = float(t)
    return best_t, best_ber

def anchored_splits(n, min_train, test_size, step):
    out = []
    tr_end = min_train
    while tr_end + test_size <= n:
        out.append((0, tr_end, tr_end, tr_end + test_size))
        tr_end += step
    return out

class DropAllNaNCols(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        X = np.asarray(X, dtype=float)
        self.keep_idx_ = np.where(~np.all(np.isnan(X), axis=0))[0]
        return self
    def transform(self, X):
        X = np.asarray(X, dtype=float)
        return X[:, self.keep_idx_]

class WelchTSelector(BaseEstimator, TransformerMixin):
    def __init__(self, k=20, eps=1e-12):
        self.k = k
        self.eps = eps
    def fit(self, X, y):
        X = np.asarray(X, dtype=float)
        y = np.asarray(y)
        pos = X[y == 1]
        neg = X[y == 0]
        m1, m0 = np.mean(pos, axis=0), np.mean(neg, axis=0)
        v1, v0 = np.var(pos, axis=0, ddof=1), np.var(neg, axis=0, ddof=1)
        n1, n0 = max(len(pos), 1), max(len(neg), 1)
        t = np.abs(m1 - m0) / (np.sqrt(v1 / n1 + v0 / n0) + self.eps)
        t = np.nan_to_num(t, nan=0.0, posinf=0.0, neginf=0.0)
        self.idx_ = np.argsort(t)[::-1][: self.k]
        return self
    def transform(self, X):
        return np.asarray(X)[:, self.idx_]

def build_pipe(method="F-test", k=20, random_state=42):
    if method == "F-test":
        sel = SelectKBest(score_func=f_classif, k=k)
    elif method == "Welch-t":
        sel = WelchTSelector(k=k)
    else:
        raise ValueError("method must be 'F-test' or 'Welch-t'")

    return Pipeline([
        ("drop_all_nan", DropAllNaNCols()),
        ("imputer", SimpleImputer(strategy="median", add_indicator=True)),
        ("scaler", StandardScaler()),
        ("sel", sel),
        ("clf", LogisticRegression(
            solver="lbfgs",
            class_weight="balanced",
            max_iter=3000,
            random_state=random_state,
        )),
    ])


# ---------- chronological data ----------
ts = pd.to_datetime(L["timestamp"], format="%d/%m/%Y %H:%M:%S", errors="coerce")
valid = ts.notna().values

Xv = X_base.iloc[valid].copy()
yv = y_bin[valid]
tv = ts.iloc[valid].reset_index(drop=True)

order = np.argsort(tv.values)
Xv = Xv.iloc[order].reset_index(drop=True)
yv = yv[order]
tv = tv.iloc[order].reset_index(drop=True)

# ---------- split config ----------
n = len(Xv)
splits = anchored_splits(
    n=n,
    min_train=int(0.50 * n),
    test_size=int(0.15 * n),
    step=max(30, int(0.15 * n) // 3),
)

# ---------- run ----------
rows = []
for split_id, (tr_s, tr_e, te_s, te_e) in enumerate(splits, start=1):
    Xtr, ytr = Xv.iloc[tr_s:tr_e], yv[tr_s:tr_e]
    Xte, yte = Xv.iloc[te_s:te_e], yv[te_s:te_e]
    ttr, tte = tv.iloc[tr_s:tr_e], tv.iloc[te_s:te_e]

    # skip pathological windows
    if len(np.unique(ytr)) < 2 or len(np.unique(yte)) < 2:
        continue

    for method in ["F-test", "Welch-t"]:
        pipe = build_pipe(method=method, k=60, random_state=42)
        pipe.fit(Xtr, ytr)

        # threshold from train probabilities only
        p_tr = pipe.predict_proba(Xtr)[:, 1]
        th, tr_ber = best_threshold_by_ber(ytr, p_tr)

        # evaluate on test at tuned threshold
        p_te = pipe.predict_proba(Xte)[:, 1]
        pred_te = (p_te >= th).astype(int)
        ber, tpr, tnr = ber_tpr_tnr(yte, pred_te)

        rows.append({
            "split_id": split_id,
            "method": method,
            "k": 20,
            "threshold": th,
            "train_BER_at_threshold": tr_ber,
            "outer_BER": ber,
            "outer_True+": tpr,
            "outer_True-": tnr,
            "train_n": len(Xtr),
            "test_n": len(Xte),
            "train_fail_rate": float(ytr.mean()),
            "test_fail_rate": float(yte.mean()),
            "train_start": ttr.min(),
            "train_end": ttr.max(),
            "test_start": tte.min(),
            "test_end": tte.max(),
        })

tuned_df = pd.DataFrame(rows)
display(tuned_df)

summary = (
    tuned_df.groupby("method")[["outer_BER", "outer_True+", "outer_True-", "threshold"]]
    .agg(["mean", "std", "median"])
)
display(summary)

pivot = tuned_df.pivot(index="split_id", columns="method", values="outer_BER")
pivot["Welch_minus_F"] = pivot["Welch-t"] - pivot["F-test"]
display(pivot)

,split_id,method,k,threshold,train_BER_at_threshold,outer_BER,outer_True+,outer_True-,train_n,test_n,train_fail_rate,test_fail_rate,train_start,train_end,test_start,test_end
0,1,F-test,20,0.34,0.181126,0.481563,0.333333,0.703540,783,235,0.085568,0.038298,2008-07-19 11:55:00,2008-09-11 07:43:00,2008-09-11 08:06:00,2008-09-22 21:31:00
1,1,Welch-t,20,0.55,0.176582,0.437316,0.333333,0.792035,783,235,0.085568,0.038298,2008-07-19 11:55:00,2008-09-11 07:43:00,2008-09-11 08:06:00,2008-09-22 21:31:00
2,2,F-test,20,0.38,0.189089,0.571115,0.142857,0.714912,861,235,0.082462,0.029787,2008-07-19 11:55:00,2008-09-16 08:50:00,2008-09-16 08:52:00,2008-09-26 02:26:00
3,2,Welch-t,20,0.47,0.197798,0.522870,0.142857,0.811404,861,235,0.082462,0.029787,2008-07-19 11:55:00,2008-09-16 08:50:00,2008-09-16 08:52:00,2008-09-26 02:26:00
4,3,F-test,20,0.43,0.200784,0.673160,0.000000,0.653680,939,235,0.080937,0.017021,2008-07-19 11:55:00,2008-09-20 05:25:00,2008-09-20 05:34:00,2008-09-29 09:15:00
5,3,Welch-t,20,0.52,0.222091,0.597403,0.000000,0.805195,939,235,0.080937,0.017021,2008-07-19 11:55:00,2008-09-20 05:25:00,2008-09-20 05:34:00,2008-09-29 09:15:00
6,4,F-test,20,0.43,0.195439,0.398539,0.363636,0.839286,1017,235,0.074730,0.046809,2008-07-19 11:55:00,2008-09-22 21:30:00,2008-09-22 21:31:00,2008-10-02 19:21:00
7,4,Welch-t,20,0.43,0.198550,0.455966,0.181818,0.906250,1017,235,0.074730,0.046809,2008-07-19 11:55:00,2008-09-22 21:30:00,2008-09-22 21:31:00,2008-10-02 19:21:00
8,5,F-test,20,0.39,0.204164,0.458446,0.294118,0.788991,1095,235,0.071233,0.072340,2008-07-19 11:55:00,2008-09-26 01:21:00,2008-09-26 02:26:00,2008-10-05 18:46:00
9,5,Welch-t,20,0.45,0.218478,0.425931,0.235294,0.912844,1095,235,0.071233,0.072340,2008-07-19 11:55:00,2008-09-26 01:21:00,2008-09-26 02:26:00,2008-10-05 18:46:00


outer_BER                     outer_True+                      \
             mean       std    median        mean       std    median   
method                                                                  
F-test   0.470594  0.127159  0.470005    0.303425  0.175867  0.322917   
Welch-t  0.469129  0.061983  0.453996    0.249609  0.139284  0.267647   

        outer_True-                     threshold                   
               mean       std    median      mean       std median  
method                                                              
F-test     0.755387  0.091222  0.719606   0.43250  0.064087   0.43  
Welch-t    0.812133  0.089834  0.808299   0.45375  0.057802   0.44

method,F-test,Welch-t,Welch_minus_F
split_id,,,
1,0.481563,0.437316,-0.044248
2,0.571115,0.522870,-0.048246
3,0.673160,0.597403,-0.075758
4,0.398539,0.455966,0.057427
5,0.458446,0.425931,-0.032515
6,0.447374,0.453939,0.006564
7,0.499001,0.454053,-0.044949
8,0.235556,0.405556,0.170000


In [26]:
# assumes X_base, y_bin, L exist and align
ts = pd.to_datetime(L["timestamp"], format="%d/%m/%Y %H:%M:%S", errors="coerce")
valid = ts.notna().values

Xv = X_base.iloc[valid].copy()
yv = y_bin[valid]
tv = ts.iloc[valid].reset_index(drop=True)

order = np.argsort(tv.values)
Xv = Xv.iloc[order].reset_index(drop=True)
yv = yv[order]
tv = tv.iloc[order].reset_index(drop=True)

cut = int(0.85 * len(Xv))  # last 15% = lockbox
X_dev, y_dev, t_dev = Xv.iloc[:cut], yv[:cut], tv.iloc[:cut]
X_lock, y_lock, t_lock = Xv.iloc[cut:], yv[cut:], tv.iloc[cut:]

print("DEV  :", len(X_dev), "fail_rate=", y_dev.mean(), t_dev.min(), "->", t_dev.max())
print("LOCK :", len(X_lock), "fail_rate=", y_lock.mean(), t_lock.min(), "->", t_lock.max())

DEV  : 1331 fail_rate= 0.07137490608564989 2008-07-19 11:55:00 -> 2008-10-05 18:59:00
LOCK : 236 fail_rate= 0.038135593220338986 2008-10-05 19:45:00 -> 2008-10-17 06:07:00


In [27]:
def anchored_splits(n, min_train, test_size, step):
    out = []
    tr_end = min_train
    while tr_end + test_size <= n:
        out.append((0, tr_end, tr_end, tr_end + test_size))
        tr_end += step
    return out


def best_threshold_by_ber(y_true, p_true, grid=None):
    if grid is None:
        grid = np.linspace(0.05, 0.95, 37)
    best_t, best_ber = 0.5, np.inf
    best_tpr, best_tnr = 0.0, 0.0
    for t in grid:
        pred = (p_true >= t).astype(int)
        ber, tpr, tnr = ber_tpr_tnr(y_true, pred)
        if (ber < best_ber) or (np.isclose(ber, best_ber) and tpr > best_tpr):
            best_t, best_ber, best_tpr, best_tnr = float(t), float(ber), float(tpr), float(tnr)
    return best_t, best_ber, best_tpr, best_tnr


def build_pipe(method, k, random_state=42):
    if method == "F-test":
        sel = SelectKBest(score_func=f_classif, k=k)
    elif method == "Welch-t":
        sel = WelchTSelector(k=k)
    else:
        raise ValueError("method must be 'F-test' or 'Welch-t'")

    return Pipeline([
        ("drop_all_nan", DropAllNaNCols()),
        ("imputer", SimpleImputer(strategy="median", add_indicator=True)),
        ("scaler", StandardScaler()),
        ("sel", sel),
        ("clf", LogisticRegression(
            solver="lbfgs",
            class_weight="balanced",
            max_iter=3000,
            random_state=random_state
        )),
    ])


# -------------------------
# 1) DEV-only time-aware tuning
# -------------------------
methods = ["F-test", "Welch-t"]
k_grid = [10, 20, 40, 60, 80]

n_dev = len(X_dev)
inner_splits = anchored_splits(
    n=n_dev,
    min_train=max(300, int(0.50 * n_dev)),
    test_size=max(120, int(0.15 * n_dev)),
    step=max(40, int(0.15 * n_dev) // 2),
)

tune_rows = []

for method in methods:
    for k in k_grid:
        fold_rows = []
        for sid, (tr_s, tr_e, va_s, va_e) in enumerate(inner_splits, start=1):
            Xtr, ytr = X_dev.iloc[tr_s:tr_e], y_dev[tr_s:tr_e]
            Xva, yva = X_dev.iloc[va_s:va_e], y_dev[va_s:va_e]

            if len(np.unique(ytr)) < 2 or len(np.unique(yva)) < 2:
                continue

            pipe = build_pipe(method=method, k=k, random_state=42)
            pipe.fit(Xtr, ytr)

            # tune threshold on this fold's train only
            p_tr = pipe.predict_proba(Xtr)[:, 1]
            th, tr_ber, _, _ = best_threshold_by_ber(ytr, p_tr)

            # evaluate on this fold's val using tuned threshold
            p_va = pipe.predict_proba(Xva)[:, 1]
            pred_va = (p_va >= th).astype(int)
            va_ber, va_tpr, va_tnr = ber_tpr_tnr(yva, pred_va)

            fold_rows.append({
                "method": method,
                "k": k,
                "split_id": sid,
                "threshold": th,
                "val_BER": va_ber,
                "val_True+": va_tpr,
                "val_True-": va_tnr,
            })

        if fold_rows:
            df = pd.DataFrame(fold_rows)
            tune_rows.append({
                "method": method,
                "k": k,
                "n_splits": len(df),
                "BER_mean": df["val_BER"].mean(),
                "BER_std": df["val_BER"].std(),
                "True+_mean": df["val_True+"].mean(),
                "True-_mean": df["val_True-"].mean(),
                "threshold_median": df["threshold"].median(),
            })

tune_summary = pd.DataFrame(tune_rows).sort_values(
    ["BER_mean", "True+_mean"], ascending=[True, False]
).reset_index(drop=True)

display(tune_summary)

best = tune_summary.iloc[0].to_dict()
print("Selected config:", best)

,method,k,n_splits,BER_mean,BER_std,True+_mean,True-_mean,threshold_median
0,F-test,10,5,0.474484,0.027524,0.220808,0.830223,0.425
1,F-test,60,5,0.482326,0.072289,0.250101,0.785247,0.400
2,Welch-t,10,5,0.485800,0.093339,0.183434,0.844966,0.425
3,Welch-t,60,5,0.485929,0.071888,0.173737,0.854405,0.425
4,F-test,40,5,0.490893,0.067248,0.190505,0.827708,0.400
5,Welch-t,20,5,0.491030,0.058379,0.216364,0.801576,0.400
6,Welch-t,80,5,0.499759,0.050552,0.180404,0.820079,0.425
7,Welch-t,40,5,0.508505,0.083470,0.107071,0.875920,0.450
8,F-test,20,5,0.513874,0.061958,0.076364,0.895889,0.450
9,F-test,80,5,0.521794,0.095969,0.234545,0.721867,0.350


Selected config: {'method': 'F-test', 'k': 10, 'n_splits': 5, 'BER_mean': 0.47448440457077545, 'BER_std': 0.027523696525041736, 'True+_mean': 0.22080808080803957, 'True-_mean': 0.8302231100504095, 'threshold_median': 0.42499999999999993}


In [28]:
# -------------------------
# 2) Fit selected config on full DEV
# -------------------------
best_method = best["method"]
best_k = int(best["k"])
best_threshold = float(best["threshold_median"])

final_pipe = build_pipe(method=best_method, k=best_k, random_state=42)
final_pipe.fit(X_dev, y_dev)

# Optional: re-tune threshold on full DEV train probs (recommended)
p_dev = final_pipe.predict_proba(X_dev)[:, 1]
best_threshold, dev_ber, dev_tpr, dev_tnr = best_threshold_by_ber(y_dev, p_dev)

print("\nFinal threshold from full DEV:", best_threshold)
print("DEV metrics @threshold:", {"BER": dev_ber, "True+": dev_tpr, "True-": dev_tnr})


Final threshold from full DEV: 0.49999999999999994
DEV metrics @threshold: {'BER': 0.28652273888605406, 'True+': 0.715789473684203, 'True-': 0.7111650485436888}


In [29]:
# -------------------------
# 3) One-time LOCKBOX evaluation
# -------------------------
p_lock = final_pipe.predict_proba(X_lock)[:, 1]
pred_lock = (p_lock >= best_threshold).astype(int)
lock_ber, lock_tpr, lock_tnr = ber_tpr_tnr(y_lock, pred_lock)

lock_result = pd.DataFrame([{
    "method": best_method,
    "k": best_k,
    "threshold": best_threshold,
    "LOCK_BER": lock_ber,
    "LOCK_True+": lock_tpr,
    "LOCK_True-": lock_tnr,
    "DEV_n": len(X_dev),
    "LOCK_n": len(X_lock),
    "DEV_fail_rate": float(np.mean(y_dev)),
    "LOCK_fail_rate": float(np.mean(y_lock)),
}])

display(lock_result)

,method,k,threshold,LOCK_BER,LOCK_True+,LOCK_True-,DEV_n,LOCK_n,DEV_fail_rate,LOCK_fail_rate
0,F-test,10,0.5,0.377386,0.333333,0.911894,1331,236,0.071375,0.038136


In [30]:
from math import comb

def bootstrap_mean_ci(x, n_boot=20000, alpha=0.05, seed=42):
    x = np.asarray(x, dtype=float)
    rng = np.random.default_rng(seed)
    boot = rng.choice(x, size=(n_boot, len(x)), replace=True).mean(axis=1)
    lo = np.percentile(boot, 100 * alpha / 2)
    md = np.percentile(boot, 50)
    hi = np.percentile(boot, 100 * (1 - alpha / 2))
    return float(lo), float(md), float(hi)

def sign_test_pvalue(d):
    # two-sided sign test for paired deltas
    d = np.asarray(d, dtype=float)
    wins = int((d < 0).sum())   # "Welch better" for BER delta if negative
    losses = int((d > 0).sum())
    n = wins + losses
    if n == 0:
        return np.nan
    k = min(wins, losses)
    p_one = sum(comb(n, i) for i in range(k + 1)) / (2 ** n)
    return min(1.0, 2 * p_one)

def paired_delta(metric, lower_is_better):
    pv = tuned_df.pivot(index="split_id", columns="method", values=metric).dropna()
    d = pv["Welch-t"] - pv["F-test"]
    # For BER: negative delta favors Welch-t
    # For True+/True-: positive delta favors Welch-t
    lo, md, hi = bootstrap_mean_ci(d.values)
    p = sign_test_pvalue(d.values if lower_is_better else -d.values)
    return pd.Series({
        "n_splits": len(d),
        "mean_delta": d.mean(),
        "median_delta": d.median(),
        "ci95_low": lo,
        "ci95_med_boot": md,
        "ci95_high": hi,
        "sign_test_pvalue": p
    })

summary = pd.DataFrame({
    "BER (Welch - F)": paired_delta("outer_BER", lower_is_better=True),
    "True+ (Welch - F)": paired_delta("outer_True+", lower_is_better=False),
    "True- (Welch - F)": paired_delta("outer_True-", lower_is_better=False),
}).T

summary

,n_splits,mean_delta,median_delta,ci95_low,ci95_med_boot,ci95_high,sign_test_pvalue
BER (Welch - F),8.0,-0.001465,-0.038381,-0.046333,-0.003472,0.056035,0.726562
True+ (Welch - F),8.0,-0.053815,0.000000,-0.142580,-0.051955,0.019717,1.000000
True- (Welch - F),8.0,0.056746,0.077730,0.005832,0.057272,0.103528,0.289062


In [31]:
tuned_df.groupby("method")[["outer_BER","outer_True+","outer_True-"]].mean()

,outer_BER,outer_True+,outer_True-
method,,,
F-test,0.470594,0.303425,0.755387
Welch-t,0.469129,0.249609,0.812133


In [32]:
# -------------------------
# 2) Fit selected config on full DEV
# -------------------------
best_method = best["method"]
best_k = int(best["k"])
best_threshold = float(best["threshold_median"])

final_pipe = build_pipe(method="Welch-t", k=best_k, random_state=42)
final_pipe.fit(X_dev, y_dev)

# Optional: re-tune threshold on full DEV train probs (recommended)
p_dev = final_pipe.predict_proba(X_dev)[:, 1]
best_threshold, dev_ber, dev_tpr, dev_tnr = best_threshold_by_ber(y_dev, p_dev)

print("\nFinal threshold from full DEV:", best_threshold)
print("DEV metrics @threshold:", {"BER": dev_ber, "True+": dev_tpr, "True-": dev_tnr})


Final threshold from full DEV: 0.44999999999999996
DEV metrics @threshold: {'BER': 0.3119826264690897, 'True+': 0.7789473684210445, 'True-': 0.5970873786407762}


In [57]:
# lockbox eval helper
def eval_lockbox(method, k, threshold):
    pipe = build_pipe(method=method, k=k, random_state=42)
    pipe.fit(X_dev, y_dev)  # fit on DEV only
    p = pipe.predict_proba(X_lock)[:, 1]
    pred = (p >= threshold).astype(int)
    ber, tpr, tnr = ber_tpr_tnr(y_lock, pred)
    return {"method": method, "k": k, "threshold": threshold,
            "LOCK_BER": ber, "LOCK_True+": tpr, "LOCK_True-": tnr}

# example thresholds from your DEV tuning medians
lock_f = eval_lockbox("F-test", 10, 0.5)   # or 0.50 if frozen earlier
lock_w = eval_lockbox("Welch-t", 10, 0.45)  # use your frozen DEV-only threshold

pd.DataFrame([lock_f, lock_w])

,method,k,threshold,LOCK_BER,LOCK_True+,LOCK_True-
0,F-test,10,0.50,0.377386,0.333333,0.911894
1,Welch-t,10,0.45,0.406021,0.333333,0.854626
